In [11]:
# ==== 0) 基本環境 ====
!nvidia-smi -L || true
!pip -q install torch torchvision timm gdown

import os, zipfile, shutil, json, math, random, time, textwrap
from pathlib import Path
from datetime import datetime

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, WeightedRandomSampler
from torchvision import datasets, transforms, models
import timm

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report, precision_recall_curve, average_precision_score

from google.colab import drive
drive.mount('/content/drive', force_remount=True)


GPU 0: Tesla T4 (UUID: GPU-0eda0d5e-eca6-0004-e2b9-1ed9ce0c0f08)
Mounted at /content/drive


In [12]:
# ==== 1) ①資料來源設定（"二選一" 取用你方便的方式）====

# 方式 A：你已把 zip 放在「我的雲端硬碟」；填寫路徑（例：/content/drive/MyDrive/leaf_cls.zip）
ZIP_PATH_IN_DRIVE = "/content/drive/MyDrive/2025_project_dataset_cls/2025_project_data_CNN_1204.zip"  # ←有就填，沒有就留空字串

# 方式 B：只有分享網址；把網址貼到這裡（支援 drive 分享連結或一般 http(s) 直連）
ZIP_FILE_URL = ""       # ←有就貼，沒有就留空字串

# (可改) 最終成果要存回雲端的資料夾
SAVE_TO_DRIVE_DIR = "/content/drive/MyDrive/Colab_Results/leaf_stage1_cls"

# (可改) 訓練常用參數
CFG = dict(
    seed=42,
    img_size=224,
    batch_size=64,           # Colab T4/RAM 通常可用 64；OOM 就降 32/16
    num_workers=2,
    epochs=25,
    lr=3e-4,
    weight_decay=1e-4,
    model_name="mobilenetv3_large_100",  # timm 名稱（或用 torchvision 官方 MobileNetV3-Large）
    mixup_p=0.0,            # 分類任務可選擇關閉
    label_smoothing=0.05,
    early_stop_patience=7
)

random.seed(CFG["seed"]); np.random.seed(CFG["seed"]); torch.manual_seed(CFG["seed"])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cuda')

In [ ]:
# ==== 2) 下載/複製 zip 到 /content，並解壓 ====
DATA_ROOT = Path("/content/dataset_zip")
DATA_ROOT.mkdir(parents=True, exist_ok=True)

zip_local = DATA_ROOT / "data.zip"

if ZIP_PATH_IN_DRIVE:
    # 從 MyDrive 複製
    src = Path(ZIP_PATH_IN_DRIVE)
    assert src.exists(), f"找不到：{src}"
    shutil.copy(str(src), str(zip_local))
    print(f"[OK] 由 Drive 複製 zip -> {zip_local}")
elif ZIP_FILE_URL:
    import gdown, re
    url = ZIP_FILE_URL.strip()
    # 支援各種 google drive 分享連結
    m = re.search(r"/d/([-\w]{20,})", url)
    if m:
        file_id = m.group(1)
        url = f"https://drive.google.com/uc?id={file_id}"
    gdown.download(url, str(zip_local), quiet=False)
    print(f"[OK] 透過連結下載 zip -> {zip_local}")
else:
    raise ValueError("請設定 ZIP_PATH_IN_DRIVE 或 ZIP_FILE_URL 其中之一。")

assert zip_local.exists(), "zip 檔案不存在"

UNZIP_DIR = DATA_ROOT / "unzipped"
if UNZIP_DIR.exists():
    shutil.rmtree(UNZIP_DIR)
UNZIP_DIR.mkdir(parents=True, exist_ok=True)

with zipfile.ZipFile(zip_local, 'r') as zf:
    zf.extractall(UNZIP_DIR)
print("[OK] 解壓完成：", UNZIP_DIR)


In [ ]:
# ==== 3) 自動偵測 train/ 與 val/ 的實際所在資料夾 ====
def find_train_val(root: Path):
    # 有些壓縮會多一層外殼資料夾；這裡嘗試往下找到 train/ val/
    candidates = [root] + [p for p in root.iterdir() if p.is_dir()]
    for base in candidates:
        t = base / "train"
        v = base / "val"
        if t.exists() and v.exists():
            return t, v
    raise RuntimeError("在解壓路徑中找不到 train/ 與 val/ 目錄")

train_dir, val_dir = find_train_val(UNZIP_DIR)
print("train_dir:", train_dir)
print("val_dir  :", val_dir)

# 檢查類別資料夾
print("\n[train 類別資料夾]")
print([p.name for p in train_dir.iterdir() if p.is_dir()])
print("[val 類別資料夾]")
print([p.name for p in val_dir.iterdir() if p.is_dir()])


In [ ]:
# ==== 4) Transforms 與 Dataset/DataLoader ====
IMG_SIZE = CFG["img_size"]

train_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.15, hue=0.02),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])

val_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])

train_ds = datasets.ImageFolder(str(train_dir), transform=train_tfms)
val_ds   = datasets.ImageFolder(str(val_dir),   transform=val_tfms)
class_names = train_ds.classes
num_classes = len(class_names)
class_names, num_classes


In [ ]:
# ==== 5) 類別不平衡處理（WeightedRandomSampler） ====
from collections import Counter

train_targets = [y for _, y in train_ds.imgs]
cnt = Counter(train_targets)
print("[train 各類別張數]：", {class_names[k]: v for k,v in cnt.items()})

# 依照 1/frequency 當作權重，緩解不平衡
sample_weights = np.array([1.0 / cnt[y] for y in train_targets], dtype=np.float32)
sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)

train_loader = DataLoader(
    train_ds, batch_size=CFG["batch_size"], sampler=sampler,
    num_workers=CFG["num_workers"], pin_memory=True
)
val_loader = DataLoader(
    val_ds, batch_size=CFG["batch_size"], shuffle=False,
    num_workers=CFG["num_workers"], pin_memory=True
)


In [ ]:
# ==== 6) 建立 MobileNetV3-Large 模型 ====
# 選用 timm 版本（速度快、預訓練權重齊全）
model = timm.create_model(CFG["model_name"], pretrained=True, num_classes=num_classes)
model.to(device)

# 損失與優化器
criterion = nn.CrossEntropyLoss(label_smoothing=CFG["label_smoothing"])
optimizer = optim.AdamW(model.parameters(), lr=CFG["lr"], weight_decay=CFG["weight_decay"])
scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())


In [ ]:
# ==== 7) 訓練/驗證 迴圈（含早停） ====
def evaluate(model, loader):
    model.eval()
    loss_sum, n = 0.0, 0
    all_probs, all_targets = [], []
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
                logits = model(x)
                loss = criterion(logits, y)
            loss_sum += loss.item() * x.size(0)
            n += x.size(0)
            probs = logits.softmax(1).detach().cpu().numpy()
            all_probs.append(probs)
            all_targets.append(y.detach().cpu().numpy())
    avg_loss = loss_sum / n
    all_probs = np.concatenate(all_probs)
    all_targets = np.concatenate(all_targets)
    preds = all_probs.argmax(1)
    acc = (preds == all_targets).mean()
    return avg_loss, acc, all_probs, all_targets

def train_epochs(model, train_loader, val_loader, epochs, save_dir):
    best_acc, best_path = -1, None
    history = {"epoch": [], "train_loss": [], "val_loss": [], "val_acc": []}
    patience = CFG["early_stop_patience"]
    wait = 0

    for epoch in range(1, epochs+1):
        model.train()
        train_loss_sum, n = 0.0, 0
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad(set_to_none=True)
            with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
                logits = model(x)
                loss = criterion(logits, y)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            train_loss_sum += loss.item() * x.size(0)
            n += x.size(0)

        train_loss = train_loss_sum / n
        val_loss, val_acc, _, _ = evaluate(model, val_loader)

        history["epoch"].append(epoch)
        history["train_loss"].append(train_loss)
        history["val_loss"].append(val_loss)
        history["val_acc"].append(float(val_acc))
        print(f"Epoch {epoch:02d}/{epochs} | train_loss {train_loss:.4f} | val_loss {val_loss:.4f} | val_acc {val_acc:.4f}")

        # 儲存最好權重 + 早停
        if val_acc > best_acc:
            best_acc = val_acc
            wait = 0
            best_path = save_dir / "best_mobilenetv3_large.pth"
            torch.save(model.state_dict(), best_path)
        else:
            wait += 1
            if wait >= patience:
                print(f"[Early Stop] patience={patience}")
                break

    # 存訓練曲線
    plt.figure(figsize=(10,4))
    plt.subplot(1,2,1); plt.plot(history["epoch"], history["train_loss"], label="train_loss"); plt.plot(history["epoch"], history["val_loss"], label="val_loss"); plt.legend(); plt.title("Loss")
    plt.subplot(1,2,2); plt.plot(history["epoch"], history["val_acc"], label="val_acc"); plt.legend(); plt.title("Val Acc")
    plt.tight_layout()
    plt.savefig(save_dir / "results_curves.png", dpi=160); plt.close()

    with open(save_dir / "history.json", "w") as f:
        json.dump(history, f, indent=2)

    return best_path, best_acc

# 建立輸出資料夾
RUN_DIR = Path("/content/runs/cls") / datetime.now().strftime("%Y%m%d_%H%M%S")
RUN_DIR.mkdir(parents=True, exist_ok=True)

best_path, best_acc = train_epochs(model, train_loader, val_loader, CFG["epochs"], RUN_DIR)
print("Best acc:", best_acc, " | best path:", best_path)


In [ ]:
# ==== 8) 使用最佳權重做完整評估（Confusion Matrix / PR 曲線 / report） ====
# 重新載入最佳權重
model.load_state_dict(torch.load(best_path, map_location=device))
val_loss, val_acc, probs, targets = evaluate(model, val_loader)
preds = probs.argmax(1)

print(f"[VALID] loss={val_loss:.4f} acc={val_acc:.4f}")
print("\n[Classification Report]\n", classification_report(targets, preds, target_names=class_names, digits=4))

# Confusion Matrix
cm = confusion_matrix(targets, preds, labels=list(range(num_classes)))
fig, ax = plt.subplots(figsize=(7,7))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
disp.plot(ax=ax, cmap="Blues", colorbar=False, xticks_rotation=45)
plt.tight_layout()
plt.savefig(RUN_DIR / "confusion_matrix.png", dpi=200); plt.close()

# PR Curves（每個類別）
plt.figure(figsize=(6,5))
for i, name in enumerate(class_names):
    y_true = (targets == i).astype(int)
    precision, recall, _ = precision_recall_curve(y_true, probs[:, i])
    ap = average_precision_score(y_true, probs[:, i])
    plt.plot(recall, precision, label=f"{name} AP={ap:.3f}")
plt.xlabel("Recall"); plt.ylabel("Precision"); plt.title("Precision-Recall Curves"); plt.legend()
plt.tight_layout()
plt.savefig(RUN_DIR / "pr_curves.png", dpi=200); plt.close()

# 也把報告文字存檔
with open(RUN_DIR / "classification_report.txt", "w") as f:
    f.write(classification_report(targets, preds, target_names=class_names, digits=4))


In [ ]:
# ==== 9) 將整包成果回存到 Google Drive ====
SAVE_DIR = Path(SAVE_TO_DRIVE_DIR) / RUN_DIR.name
SAVE_DIR.parent.mkdir(parents=True, exist_ok=True)

# 拷貝結果
if SAVE_DIR.exists():
    shutil.rmtree(SAVE_DIR)
shutil.copytree(RUN_DIR, SAVE_DIR)

print(f"[OK] 已將結果複製到：{SAVE_DIR}")

# 額外：也把當次使用的 zip 與「偵測到的 train/val 結構」記錄備查
shutil.copy(str(zip_local), str(SAVE_DIR / "source_dataset.zip"))
with open(SAVE_DIR / "dataset_paths.txt", "w") as f:
    f.write(f"train_dir: {train_dir}\nval_dir  : {val_dir}\nclasses  : {class_names}\n")
print("[DONE]")


In [15]:
# === Re-run only：用已存的最佳權重 + 重新解壓資料集，做完整評估與逐檔 CSV ===
!pip -q install torch torchvision timm
import os, re, json, zipfile, shutil, glob
from pathlib import Path
import numpy as np
import torch, torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import timm
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report, precision_recall_curve, average_precision_score

from google.colab import drive
drive.mount("/content/drive", force_remount=True)

# ==== 1) 指定「最佳 run 資料夾」：請把這一行改成你上次結果的資料夾 ====
BEST_RUN_DIR = Path("/content/drive/MyDrive/Colab_Results/leaf_stage1_cls/20251204")  # ←改成你的資料夾名稱

# 讀最佳權重（預設我上次存成 best_mobilenetv3_large.pth，也容許你換檔名）
best_weight = next(iter([*BEST_RUN_DIR.glob("best_*.pth"), *BEST_RUN_DIR.glob("*.pth")]))
print("[BEST RUN]", BEST_RUN_DIR)
print("[BEST WEIGHT]", best_weight.name)

# ==== 2) 重新解壓資料集（用 run 內的 source_dataset.zip） ====
zip_path = BEST_RUN_DIR / "source_dataset.zip"
assert zip_path.exists(), f"找不到 source_dataset.zip：{zip_path}"

UNZIP_DIR = Path("/content/dataset_zip/unzipped")
if UNZIP_DIR.exists():
    shutil.rmtree(UNZIP_DIR)
UNZIP_DIR.mkdir(parents=True, exist_ok=True)

with zipfile.ZipFile(zip_path, "r") as zf:
    zf.extractall(UNZIP_DIR)

# 自動往下找 train/ 與 val/（有時 zip 會多一層資料夾殼）
def find_train_val(root: Path):
    cand = [root] + [p for p in root.iterdir() if p.is_dir()]
    for base in cand:
        t, v = base / "train", base / "val"
        if t.exists() and v.exists():
            return t, v
    raise RuntimeError("在解壓路徑中找不到 train/ 與 val/ 目錄")

train_dir, val_dir = find_train_val(UNZIP_DIR)
print("train_dir :", train_dir)
print("val_dir   :", val_dir)

# 類別名以「val 子資料夾」為準（避免名字不同步）
class_names = sorted([p.name for p in val_dir.iterdir() if p.is_dir()])
num_classes = len(class_names)
print("classes  :", class_names)

# ==== 3) 建立 DataLoader（只做驗證，不需要 sampler/增強） ====
IMG_SIZE = 224
BATCH    = 64

val_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])
val_ds = datasets.ImageFolder(str(val_dir), transform=val_tfms)
val_loader = DataLoader(val_ds, batch_size=BATCH, shuffle=False, num_workers=2, pin_memory=True)

# ==== 4) 建立同架構模型並載入最佳權重 ====
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = timm.create_model("mobilenetv3_large_100", pretrained=False, num_classes=num_classes)
model.load_state_dict(torch.load(best_weight, map_location=device))
model.to(device)
model.eval()

# ==== 5) 做完整評估（報表、混淆矩陣、PR 曲線） ====
all_probs, all_targets = [], []
with torch.no_grad():
    for x, y in val_loader:
        x = x.to(device)
        logits = model(x)
        probs = logits.softmax(1).cpu().numpy()
        all_probs.append(probs)
        all_targets.append(y.numpy())
probs   = np.concatenate(all_probs)
targets = np.concatenate(all_targets)
preds   = probs.argmax(1)

# 報表
rep_txt = classification_report(targets, preds, target_names=class_names, digits=4)
print(rep_txt)
(BEST_RUN_DIR / "reval").mkdir(exist_ok=True)
with open(BEST_RUN_DIR / "reval" / "classification_report_rerun.txt", "w") as f:
    f.write(rep_txt)

# 混淆矩陣
cm = confusion_matrix(targets, preds, labels=list(range(num_classes)))
fig, ax = plt.subplots(figsize=(7,7))
ConfusionMatrixDisplay(cm, display_labels=class_names).plot(ax=ax, cmap="Blues", colorbar=False, xticks_rotation=45)
plt.tight_layout()
plt.savefig(BEST_RUN_DIR / "reval" / "confusion_matrix_rerun.png", dpi=200); plt.close()

# PR Curves
plt.figure(figsize=(6,5))
for i, name in enumerate(class_names):
    y_true = (targets == i).astype(int)
    from sklearn.metrics import precision_recall_curve, average_precision_score
    precision, recall, _ = precision_recall_curve(y_true, probs[:, i])
    ap = average_precision_score(y_true, probs[:, i])
    plt.plot(recall, precision, label=f"{name} AP={ap:.3f}")
plt.xlabel("Recall"); plt.ylabel("Precision"); plt.title("Precision-Recall Curves (rerun)"); plt.legend()
plt.tight_layout()
plt.savefig(BEST_RUN_DIR / "reval" / "pr_curves_rerun.png", dpi=200); plt.close()

# ==== 6) 輸出「逐檔預測」CSV（含你要找的「others → pepper_bell」） ====
import csv

inv_cls = {i:c for i,c in enumerate(class_names)}
rows = []
for idx, (path, true_idx) in enumerate(val_ds.imgs):
    pred_idx = int(preds[idx])
    rows.append({
        "filepath": path,
        "true_label": inv_cls[int(true_idx)],
        "pred_label": inv_cls[pred_idx],
        "correct"  : int(pred_idx == true_idx),
        **{f"prob_{inv_cls[i]}": float(probs[idx, i]) for i in range(num_classes)}
    })

csv_path = BEST_RUN_DIR / "reval" / "val_predictions_rerun.csv"
with open(csv_path, "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=list(rows[0].keys()))
    writer.writeheader(); writer.writerows(rows)
print("[CSV] 全部驗證檔預測已輸出：", csv_path)

# 只輸出「others → pepper_bell」的誤判清單
try:
    others_idx      = class_names.index("others")
    pepper_bell_idx = class_names.index("pepper_bell")
    bad_rows = [r for r in rows if (r["true_label"]=="others" and r["pred_label"]=="pepper_bell")]
    csv_bad = BEST_RUN_DIR / "reval" / "miscls_others_to_pepper_bell.csv"
    with open(csv_bad, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=list(rows[0].keys()))
        writer.writeheader(); writer.writerows(bad_rows)
    print(f"[CSV] others→pepper_bell 誤判清單共 {len(bad_rows)} 筆：", csv_bad)
except ValueError:
    print("注意：找不到 'others' 或 'pepper_bell' 類別名稱，請確認資料夾命名。")


Mounted at /content/drive
[BEST RUN] /content/drive/MyDrive/Colab_Results/leaf_stage1_cls/20251204
[BEST WEIGHT] best_mobilenetv3_large.pth
train_dir : /content/dataset_zip/unzipped/2025_project_data_CNN_1204/train
val_dir   : /content/dataset_zip/unzipped/2025_project_data_CNN_1204/val
classes  : ['others', 'pepper_bell', 'potato', 'tomato', 'whole_plant']
              precision    recall  f1-score   support

      others     1.0000    0.8940    0.9440       500
 pepper_bell     0.9357    1.0000    0.9668       495
      potato     0.9795    0.9977    0.9885       431
      tomato     0.9842    0.9960    0.9901       500
 whole_plant     0.9906    1.0000    0.9953       527

    accuracy                         0.9772      2453
   macro avg     0.9780    0.9775    0.9769      2453
weighted avg     0.9782    0.9772    0.9768      2453

[CSV] 全部驗證檔預測已輸出： /content/drive/MyDrive/Colab_Results/leaf_stage1_cls/20251204/reval/val_predictions_rerun.csv
[CSV] others→pepper_bell 誤判清單共 33 筆： /c